In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
from util import bullish_sequence, bullish_candlestick_patterns
from src.asset_selector import AssetSelector
from src.indicators import Indicators
from src.predictor import Predictor
from sklearn.cluster import KMeans
import alpaca_trade_api as tradeapi
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.offline as py
import configparser
import pandas as pd
import numpy as np
import sys
import os

py.init_notebook_mode(connected=True)
config = configparser.ConfigParser()

try:
    config.read(os.path.relpath("config.ini"))
except FileExistsError as e:
    print("FileExistsError: {}".format(e))
    sys.exit(1)


alpaca_api = tradeapi.REST(
    base_url    = config["alpaca"]["APCA_API_BASE_URL"],
    key_id      = config["alpaca"]["APCA_API_KEY_ID"],
    secret_key  = config["alpaca"]["APCA_API_SECRET_KEY"],
    api_version = config["alpaca"]["VERSION"]
)

In [ ]:
assets      = AssetSelector(alpaca_api, edgar_token=None).get_assets_by_candlestick_pattern()
indicators  = Indicators(alpaca_api, assets).get_cluster()

In [ ]:
predictions = Predictor(indicators)

In [ ]:
indicators["GEF.B"].iloc[-20:]

In [ ]:
kmeans = KMeans(n_clusters=2)

In [ ]:
n_indicators            = pd.DataFrame()
n_indicators["close"]   = indicators["GEF.B"]["close"].iloc[-20:]
n_indicators["macd"]    = indicators["GEF.B"]["macd"].iloc[-20:]
n_indicators["signal"]  = indicators["GEF.B"]["signal"].iloc[-20:]
n_indicators["mfi"]     = indicators["GEF.B"]["mfi"].iloc[-20:]
n_indicators["bb_up"]   = indicators["GEF.B"]["bb_up"].iloc[-20:]
n_indicators["bb_low"]  = indicators["GEF.B"]["bb_low"].iloc[-20:]
n_indicators["vzo"]     = indicators["GEF.B"]["vzo"].iloc[-20:]
n_indicators["apz_u"]   = indicators["GEF.B"]["apz_u"].iloc[-20:]
n_indicators["apz_l"]   = indicators["GEF.B"]["apz_l"].iloc[-20:]
n_indicators["stoch"]   = indicators["GEF.B"]["stoch"].iloc[-20:]

In [ ]:
kmeans.fit(n_indicators)

In [ ]:
kmeans.cluster_centers_

In [ ]:
kmeans.labels_

In [ ]:
f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, sharex=True, sharey=False, figsize=(10,6))
ax1.set_title('K Means - macd/close')
ax1.scatter(n_indicators["macd"], n_indicators["close"], c=kmeans.labels_, cmap="rainbow")
ax2.set_title("K Means - signal/close")
ax2.scatter(n_indicators["signal"], n_indicators["close"], c=kmeans.labels_, cmap="rainbow")
ax3.set_title("Original")
ax3.scatter(n_indicators["macd"], n_indicators["close"], c=n_indicators["macd"], cmap="rainbow")
ax4.set_title("Original")
ax4.scatter(n_indicators["signal"], n_indicators["close"], c=n_indicators["signal"], cmap="rainbow")

In [ ]:
f, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, sharex=True, sharey=True, figsize=(10,6))
ax1.set_title('K Means - macd/close')
ax1.scatter(n_indicators["macd"], n_indicators["close"], c=kmeans.labels_, cmap="rainbow")
ax2.set_title("K Means - signal/close")
ax2.scatter(n_indicators["signal"], n_indicators["close"], c=kmeans.labels_, cmap="rainbow")
ax3.set_title("Original")
ax3.scatter(n_indicators["macd"], n_indicators["close"], c=n_indicators["close"], cmap="rainbow")
ax4.set_title("Original")
ax4.scatter(n_indicators["signal"], n_indicators["close"], c=n_indicators["close"], cmap="rainbow")

In [ ]:
f, (ax3, ax4) = plt.subplots(1, 2, sharex=True, sharey=True,figsize=(10,6))
ax3.set_title('K Means - mfi/close')
ax3.scatter(n_indicators["mfi"], n_indicators["close"], c=kmeans.labels_, cmap="rainbow")
ax4.set_title("Original")
ax4.scatter(n_indicators["mfi"], n_indicators["close"], c=n_indicators["close"], cmap="rainbow")

In [ ]:
f, (ax5, ax6) = plt.subplots(1, 2, sharex=True, sharey=True,figsize=(10,6))
ax5.set_title('K Means - bb_low/mfi')
ax5.scatter(n_indicators["bb_low"], n_indicators["mfi"], c=kmeans.labels_, cmap="rainbow")
ax6.set_title("Original")
ax6.scatter(n_indicators["bb_low"], n_indicators["mfi"], c=n_indicators["close"], cmap="rainbow")

In [ ]:
f, (ax7, ax8) = plt.subplots(1, 2, sharex=True, sharey=True,figsize=(10,6))
ax7.set_title('K Means - vzo/close')
ax7.scatter(n_indicators["vzo"], n_indicators["close"], c=kmeans.labels_, cmap="rainbow")
ax8.set_title("Original")
ax8.scatter(n_indicators["vzo"], n_indicators["close"], c=n_indicators["close"], cmap="rainbow")

In [ ]:
f, (ax9, ax10) = plt.subplots(1, 2, sharex=True, sharey=True,figsize=(10,6))
ax9.set_title('K Means - vzo/mfi')
ax9.scatter(n_indicators["vzo"], n_indicators["mfi"], c=kmeans.labels_, cmap="rainbow")
ax10.set_title("Original")
ax10.scatter(n_indicators["vzo"], n_indicators["mfi"], c=n_indicators["close"], cmap="rainbow")

In [ ]:
heatmap = go.Heatmap(
    z=n_indicators.iloc[-20:].corr(method='pearson').values,
    x=n_indicators.columns,
    y=n_indicators.columns,
    colorbar=dict(title='Pearson Coefficient'),
)

layout = go.Layout(title="Pearson correlation of indicators")

heatmap['zmax'] = 1.0
heatmap['zmin'] = -1.0

fig = go.Figure(data=[heatmap], layout=layout)
py.iplot(fig)

In [ ]:
heatmap = go.Heatmap(
    z=n_indicators.iloc[-20:].pct_change().corr(method='pearson').values,
    x=n_indicators.columns,
    y=n_indicators.columns,
    colorbar=dict(title='Pearson Coefficient'),
)

layout = go.Layout(title="Pearson correlation of indicator percentage changes")

heatmap['zmax'] = 1.0
heatmap['zmin'] = -1.0

fig = go.Figure(data=[heatmap], layout=layout)
py.iplot(fig)

In [ ]:
n_indicators            = pd.DataFrame()
n_indicators["close"]   = indicators["GEF.B"]["close"]
n_indicators["macd"]    = indicators["GEF.B"]["macd"]
n_indicators["signal"]  = indicators["GEF.B"]["signal"]
n_indicators["mfi"]     = indicators["GEF.B"]["mfi"]
n_indicators["bb_up"]   = indicators["GEF.B"]["bb_up"]
n_indicators["bb_low"]  = indicators["GEF.B"]["bb_low"]
n_indicators["vzo"]     = indicators["GEF.B"]["vzo"]
n_indicators["apz_u"]   = indicators["GEF.B"]["apz_u"]
n_indicators["apz_l"]   = indicators["GEF.B"]["apz_l"]
n_indicators["stoch"]   = indicators["GEF.B"]["stoch"]

# train models
close_train, close_test     = predictions.get_predictions(n_indicators["close"])
macd_train, macd_test       = predictions.get_predictions(n_indicators["macd"])
signal_train, signal_test   = predictions.get_predictions(n_indicators["signal"])
mfi_train, mfi_test         = predictions.get_predictions(n_indicators["mfi"])
stoch_train, stoch_test     = predictions.get_predictions(n_indicators["stoch"])
vzo_train, vzo_test         = predictions.get_predictions(n_indicators["vzo"])

In [ ]:
close_test      = close_test[~np.isnan(close_test)]
macd_test       = macd_test[~np.isnan(macd_test)]
signal_test     = signal_test[~np.isnan(signal_test)]
mfi_test        = mfi_test[~np.isnan(mfi_test)]
stoch_test      = stoch_test[~np.isnan(stoch_test)]
vzo_test        = vzo_test[~np.isnan(vzo_test)]

In [ ]:
# close
plt.plot(np.cumprod(close_train+1), label="ass")
plt.plot(np.cumprod(close_test+1), label="dick")

In [ ]:
# macd
plt.plot(np.cumprod(macd_train+1), color="green")
plt.plot(np.cumprod(signal_train+1), color="orange")
# test
plt.plot(np.cumprod(macd_test+1), color="blue")
plt.plot(np.cumprod(signal_test+1), color="red")

In [ ]:
# MFI
plt.plot(np.cumprod(mfi_train+1))
plt.plot(np.cumprod(mfi_test+1))

In [ ]:
# stochastic oscillator  
plt.plot(np.cumprod(stoch_train+1))
plt.plot(np.cumprod(stoch_test+1))

In [ ]:
# macd and signal + close w/ volume
plt.plot(np.cumprod(close_test+1), color="blue")
plt.plot(np.cumprod(macd_test+1), color="orange")
plt.plot(np.cumprod(signal_test+1), color="green")
# plt.plot(np.cumprod(volume_test+1), color="black")

In [ ]:
# stoch + mfi w/ volume
plt.plot(np.cumprod(mfi_test+1), color="#9467bd")
plt.plot(np.cumprod(stoch_test+1), color="#17becf")
# plt.plot(np.cumprod(volume_test+1), color="black")

In [ ]:
print("close test:", close_test[-1], " current close:", n_indicators["close"].iloc[-1])
print("macd test:", macd_test[-1], " current macd:", n_indicators["macd"].iloc[-1])
print("signal test:", signal_test[-1], " current signal:", n_indicators["signal"].iloc[-1])
print("mfi test:", mfi_test[-1], " current mfi:", n_indicators["mfi"].iloc[-1])
print("stoch test:", stoch_test[-1], " current stoch:", n_indicators["stoch"].iloc[-1])
print("VZO test:", vzo_test[-1], " current VZO:", n_indicators["vzo_test"].iloc[-1])

In [ ]:
# print("sentiment:", sentiment)


In [ ]:
print(n_indicators["macd"].iloc[-1] < n_indicators["signal"].iloc[-1])

In [ ]:
print(bullish_sequence(n_indicators["macd"].iloc[-5], n_indicators["macd"].iloc[-4], n_indicators["macd"].iloc[-3]))

In [ ]:
pattern = bullish_candlestick_patterns(n_indicators.iloc[-1], n_indicators.iloc[-2], n_indicators.iloc[-3])
print("Pattern: ", pattern, pattern in ["hammer", "inverseHammer"])